<a href="https://colab.research.google.com/github/PawarKishori/Text-Summarization/blob/main/Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
pip install datasets evaluate rouge_score
! pip install -U accelerate
! pip install -U transformers

SyntaxError: invalid syntax (<ipython-input-2-77e0733c39db>, line 1)

In [3]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

In [5]:
billsum = billsum.train_test_split(test_size=0.2)

In [6]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\n(a) The Legislature finds and declares all of the following:\n(1) Park access, outdoor education, and outdoor recreational experiences are important to the health and well-being of all California citizens as well as the continuing stewardship of our natural resources. Many California communities, however, lack equitable access to parks and other open-space areas. This lack of access to the outdoors contributes to higher incidences of certain health ailments, such as diabetes, hypertension,\nobesity,\nand nature deficit disorder, and other negative social indicators, especially in low-income communities.\n(2) According to the recent Parks Forward Commission report,\nreleased in February 2015,\nimproving access to parks, outdoor experiences, and recreational opportunities, particularly for youth and young adults in disadvantaged communities, will lead to more healthy lifestyles, better educational outco

In [7]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [11]:
import evaluate

rouge = evaluate.load("rouge")

In [12]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [13]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.873695,0.124400,0.031600,0.103900,0.103700,19.000000
2,No log,2.659910,0.135600,0.045700,0.114400,0.114200,19.000000
3,No log,2.596929,0.137400,0.045000,0.114500,0.114400,19.000000
4,No log,2.580513,0.137900,0.045700,0.115100,0.115200,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=248, training_loss=3.0141279159053678, metrics={'train_runtime': 284.5399, 'train_samples_per_second': 13.903, 'train_steps_per_second': 0.872, 'total_flos': 1070824333246464.0, 'train_loss': 3.0141279159053678, 'epoch': 4.0})

In [17]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1708184383.208e938202fa.6690.0:   0%|          | 0.00/7.79k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PawarKishori/my_awesome_billsum_model/commit/e04b60da0fdba989b03cf9c6633b8cfb622811c2', commit_message='End of training', commit_description='', oid='e04b60da0fdba989b03cf9c6633b8cfb622811c2', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [19]:
from transformers import pipeline

summarizer = pipeline("summarization", model="PawarKishori/my_awesome_billsum_model")
summarizer(text)

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Your max_length is set to 200, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


[{'summary_text': "the Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country."}]

In [22]:
healthcare_text='''
Most modern NLP applications use state-of-the-art deep learning methods. You can read more about the NLP types and approaches in our dedicated article. But what you need to know here is that deep learning or deep neural networks can understand and analyze data with minimum preprocessing. They’re smart enough to independently perform different NLP processes. These tasks include:
Text classification -- categorizing unstructured data. For example, organizing patient application forms by urgency or pinpointing fraudulent claims.
Information extraction -- retrieving valuable information from unstructured data. Say, the system can tag data from patient history, discharge summary, or call center reports and then structure them in an EHR according to a schema.
Language modeling -- understanding spoken text and producing natural sounding text. The software can accurately transcribe physician notes and then summarize them or further classify and extract data.
How can NLP benefit healthcare organizations?
Natural language processing offers opportunities to tackle several problems physicians, patients, and clinics have.

Alleviate clinician burnout. Physician productivity and motivation suffer from the glut of repetitive administrative tasks that force them to spend extra hours at the computer instead of interacting with patients. This problem even has a name -- the EHR burden. NLP offers several solutions to help doctors from speech-to-text transcribing technology to simplified clinical documentation management.

Streamline administrative processes. Such admin tasks as prior authorization of a patient’s health plan and claims processing contribute to the aforementioned burnout and high billing and insurance-related (BIR) costs. By recent estimations, BIR expenses for healthcare providers range from $20 for a primary care visit to $215 for an inpatient surgical procedure. NLP-powered systems can automate many of the steps in claim filing and reduce turnaround time.

Enhance clinical decision support. Physicians have been using software for making informed care decisions for years now. Typically installed as a part of an EHR, clinical decision support systems (CDSSs) provide helpful prompts on drug selection, diagnostics, and other actions by automatically checking knowledge bases and comparing them with patient records. In some cases, the use of a CDSS is even obligatory, for example, when ordering expensive tests to check that the service provider won’t receive reimbursement for it. However useful, CDSSs are mostly limited to processing only structured data. NLP can provide way more information for a CDSS from sources that it wouldn’t use otherwise and power predictive analytics.

Improve patient interaction and engagement. Patients’ engagement in their own care is one of the key factors in successful health outcomes. So, healthcare providers must actively invite patients to educate themselves, take part in decision-making, and reach out to their physicians when required. NLP has tons of applications to communicate with patients (via automatic calls and chatbots), improve their health comprehension (by estimating their health literacy profiles and replacing medical jargon), and provide simple ways to share their health information, for example, through voice messages.
'''


In [23]:
summarizer(healthcare_text)

[{'summary_text': 'NLP offers several solutions to help physicians from speech-to-text transcribing technology to simplified clinical documentation management . Natural language processing offers opportunities to tackle several problems physicians, patients, and clinics have . NLP-powered systems can automate many of the steps in claim filing and reduce turnaround time.'}]